In [1]:
# Imports
import copy
import os
import pandas as pd
import pickle
import yaml

In [2]:
def color_strategy(val):
    """Return a CSS background-color depending on the action."""
    if val == "S":
        return "background-color: gold"     
    elif val == "H":
        return "background-color: white"    
    else:
        return "" 

In [3]:
directory = "../dicts/"
dicts = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):  # or any extension your pickle files use
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            d = pickle.load(f)
            dicts.append(d)

results = {}
for d in dicts:
    for key, inner in d.items():
        if key not in results:
            results[key] = {}
        for subkey, sublist in inner.items():
            if subkey not in results[key]:
                results[key][subkey] = []
            results[key][subkey].extend(sublist)

In [4]:
# Convert lists into letters
results_copy = copy.deepcopy(results)
for row in results.items():
    for key in row[1].keys():
        row[1][key] = 'H' if round(sum(row[1][key]) / len(row[1][key])) == 0 else 'S'

In [5]:
# Visualize results
df = pd.DataFrame.from_dict(results, orient='index')
styled_df = df.style.map(color_strategy)
styled_df

,2,3,4,5,6,7,8,9,10,A
17,S,S,S,S,S,S,H,H,H,H
16,H,H,H,S,S,H,H,H,H,H
15,H,H,H,H,S,H,H,H,H,H
14,H,H,H,H,H,H,H,H,H,H
13,H,H,H,H,H,H,H,H,H,H
12,H,H,H,H,S,H,H,H,H,H
11,H,H,H,H,H,H,H,H,H,H
10,H,H,H,H,H,H,H,H,H,H
9,H,H,H,H,H,H,H,H,H,H
8,H,H,H,H,H,H,H,H,H,H


In [2]:
# Report winrate

import os
def calculate_winrate(directory):
    won = 0
    total_games = 0

    z = 1.96

    for filename in os.listdir(directory):
        if filename.endswith(".yaml"):
            filepath = os.path.join(directory, filename)

            with open(filepath, "r") as file:
                data = yaml.safe_load(file)

                if "final results" in data[-1]:
                    total_games += 1
                    if str(data[-1]["final results"]).strip().lower() == "win.":
                        won += 1
                    elif str(data[-1]["final results"]).strip().lower() == "draw.":
                        won += 0.5
                else:
                    os.remove(filepath)

    winrate = (won / total_games) if total_games > 0 else 0
    margin = (z * ((winrate * (1 - winrate) / total_games) ** 0.5)) * 100
    winrate *= 100
    return won, total_games, winrate, margin

In [5]:
directory_path = "../Agent-Pro/my_data/31_run"
wins, total, winrate, margin = calculate_winrate(directory_path)

print(f"Total Wins: {wins}")
print(f"Total Games: {total}")
print(f"Win Rate: {winrate:.2f}% ± {margin:.2f}%")

Total Wins: 306.0
Total Games: 818
Win Rate: 37.41% ± 3.32%
